In [1]:
from sagemaker import get_execution_role

In [2]:
custom_code_upload_location = 's3://sagemaker-02122018/customcode/tensorflow_iris'

In [3]:
model_artifacts_location = 's3://sagemaker-02122018/artifacts'

In [4]:
role = get_execution_role()

In [5]:
!cat "iris_dnn_1.py" #successful training but not compilation, "UnexpectedStatusException: Error for Compilation job compilation-sagemaker-tensorflow-2020-09-25-15-34-23-845: Failed. Reason: ClientError: InputConfiguration: TVM cannot convert Tensorflow model. Please make sure the framework you selected is correct. The following operators are not implemented: {'AsString'}"

from __future__ import absolute_import

import numpy as np
import os
import tensorflow as tf

INPUT_TENSOR_NAME = 'inputs'
input_tensor_name = 'inputs'

def estimator_fn(run_config, params): #Function 1 Estimator 
    feature_columns = [tf.feature_column.numeric_column(INPUT_TENSOR_NAME, shape=[4])] #shape represents the four elements of the data set 
    return tf.estimator.DNNClassifier(feature_columns=feature_columns,#s
                                      hidden_units=[10, 20, 10], #Deep Neural Network Classifier
                                      n_classes=3, #Three output classes because we have three species of flowers
                                      config=run_config)


def serving_input_fn(params):  #Function 2 Once the model is deployed how model is going to get the inputs 
    feature_spec = {INPUT_TENSOR_NAME: tf.placeholder(tf.float32,[None,4],name=None)}
    return tf.estimator.export.ServingInputReceiver(feature_spec,feature_spec)


def train_input_fn(training_

In [6]:
from sagemaker.tensorflow import TensorFlow

In [7]:
iris_estimator = TensorFlow(entry_point='iris_dnn_1.py',  #Creating Estimator,etreing python scrypt
                            role=role, #IAM Role
                            framework_version='1.11.0',
                            output_path=model_artifacts_location,
                            code_location=custom_code_upload_location,
                            train_instance_count=1, #no of instences
                            train_instance_type='ml.c4.xlarge',
                            training_steps=1000,  
                            evaluation_steps=100)


2.1.0 is the latest version of tensorflow that supports Python 2. Newer versions of tensorflow will only be available for Python 3.Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.


In [8]:
%%time
import boto3

region = boto3.Session().region_name
train_data_location = 's3://sagemaker-sample-data-{}/tensorflow/iris'.format(region)

iris_estimator.fit(train_data_location)


'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-09-26 15:36:36 Starting - Starting the training job...
2020-09-26 15:36:38 Starting - Launching requested ML instances......
2020-09-26 15:37:47 Starting - Preparing the instances for training......
2020-09-26 15:39:06 Downloading - Downloading input data...
2020-09-26 15:39:18 Training - Downloading the training image..2020-09-26 15:39:38,299 INFO - root - running container entrypoint
2020-09-26 15:39:38,300 INFO - root - starting train task
2020-09-26 15:39:38,313 INFO - container_support.training - Training starting
2020-09-26 15:39:41,664 INFO - tf_container - ----------------------TF_CONFIG--------------------------
2020-09-26 15:39:41,664 INFO - tf_container - {"environment": "cloud", "cluster": {"master": ["algo-1:2222"]}, "task": {"index": 0, "type": "master"}}
2020-09-26 15:39:41,664 INFO - tf_container - ---------------------------------------------------------
2020-09-26 15:39:41,664 INFO - tf_container - creating RunConfig:
2020-09-26 15:39:41,664 INFO - tf_container -


2020-09-26 15:40:05 Uploading - Uploading generated training model
2020-09-26 15:40:05 Completed - Training job completed
2020-09-26 15:39:51,426 INFO - tensorflow - Restoring parameters from s3://sagemaker-02122018/artifacts/sagemaker-tensorflow-2020-09-26-15-36-36-220/checkpoints/model.ckpt-1000
2020-09-26 15:39:51,581 INFO - tensorflow - Running local_init_op.
2020-09-26 15:39:51,593 INFO - tensorflow - Done running local_init_op.
2020-09-26 15:39:51,721 INFO - tensorflow - Evaluation [10/100]
2020-09-26 15:39:51,740 INFO - tensorflow - Evaluation [20/100]
2020-09-26 15:39:51,759 INFO - tensorflow - Evaluation [30/100]
2020-09-26 15:39:51,779 INFO - tensorflow - Evaluation [40/100]
2020-09-26 15:39:51,797 INFO - tensorflow - Evaluation [50/100]
2020-09-26 15:39:51,814 INFO - tensorflow - Evaluation [60/100]
2020-09-26 15:39:51,830 INFO - tensorflow - Evaluation [70/100]
2020-09-26 15:39:51,848 INFO - tensorflow - Evaluation [80/100]
2020-09-26 15:39:51,868 INFO - tensorflow - Evalu

Training seconds: 59
Billable seconds: 59
CPU times: user 560 ms, sys: 45.1 ms, total: 606 ms
Wall time: 3min 41s


In [9]:
# %%time

# iris_predictor = iris_estimator.deploy(initial_instance_count=1,
#                                        instance_type='ml.c4.xlarge')


In [10]:
# iris_predictor.predict([7.0, 3.2, 4.7, 1.4])

In [11]:
output_path = '/'.join(iris_estimator.output_path.split('/')[:-1])
optimized_estimator = iris_estimator.compile_model(target_instance_family='rasp3b', 
                              input_shape={'data':[1, 4]},  # Batch size 1, 3 channels, 224x224 Images.
                              output_path=output_path,
                              framework='tensorflow', framework_version='1.11.0')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
2.1.0 is the latest version of tensorflow that supports Python 2. Newer versions of tensorflow will only be available for Python 3.Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


?...*

UnexpectedStatusException: Error for Compilation job compilation-sagemaker-tensorflow-2020-09-26-15-40-18-208: Failed. Reason: ClientError: InputConfiguration: TVM cannot convert Tensorflow model. Please make sure the framework you selected is correct. The following operators are not implemented: {'AsString'}